In [1]:
import sqlite3
import csv

In [3]:
# Define tables for the SQL database
nodes = '''CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);'''

nodes_tags = '''CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);'''

ways = '''CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);'''

ways_tags = '''CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);'''

ways_nodes = '''CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);'''

In [4]:
# Create the SQL database
conn = sqlite3.connect('OSM_cuxhaven.db')

In [5]:
# Create tables
cursor = conn.cursor()
cursor.execute(nodes)
cursor.execute(nodes_tags)
cursor.execute(ways)
cursor.execute(ways_tags)
cursor.execute(ways_nodes)
conn.commit()

In [2]:
NODES_PATH = "21_nodes.csv"
NODE_TAGS_PATH = "22_nodes_tags.csv"
WAYS_PATH = "23_ways.csv"
WAY_NODES_PATH = "24_ways_nodes.csv"
WAY_TAGS_PATH = "25_ways_tags.csv"

In [ ]:
# Populate the database tables with data from CSVs
# Read in the csv file as a dictionary and formatting the data as a list of tuples,
# then insert the formatted data in the database table 

# populating the nodes_tags table
with open('22_nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type']) for i in dr]
    
cursor.executemany("INSERT INTO nodes_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()

# populating the ways table
with open('23_ways.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['user'].decode("utf-8"), i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cursor.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

# populating the nodes table
with open('21_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['id'], i['lat'], i['lon'], i['user'].decode("utf-8"), i['uid'], i['version'], i['changeset'], i['timestamp']) 
             for i in dr]

cursor.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", 
                   to_db)
conn.commit()

# populating the ways_tags table
with open('25_ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type']) for i in dr]
    
cursor.executemany("INSERT INTO ways_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()

# populating the ways_nodes table 
with open('24_ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]
    
cursor.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db)
conn.commit()
conn.close()

'''
cursor.execute('SELECT * FROM nodes_tags')
all_rows = cursor.fetchall()
print('1):')
pprint(all_rows[:20])
'''